In [1]:
import requests,pprint,json,datetime,time
from gzip import decompress
from json import loads
from requests import get
import pandas as pd
from pymongo import MongoClient
from APIweatherkey import key 
import numpy as np
#key = 'c1a5f0a0817c9b0d547ef47d4873d62c'
# Sign up for free API key at https://openweathermap.org/appid

ModuleNotFoundError: No module named 'APIweatherkey'

# Acquring data from an API

In [2]:
city='vancouver'
country='ca'


requestString='https://api.openweathermap.org/data/2.5/weather?q=%s,%s&APPID=%s' % (city,country,key)


res=requests.get(requestString)
print(res.status_code)

200


In [5]:
type(res)

requests.models.Response

In [7]:
mydict = res.json()

In [14]:
mydict['weather'][0]['main']

'Clouds'

In [132]:
mydict

{'coord': {'lon': -124.26, 'lat': 54.46},
 'weather': [{'id': 804,
   'main': 'Clouds',
   'description': 'overcast clouds',
   'icon': '04n'}],
 'base': 'stations',
 'main': {'temp': 282.64,
  'feels_like': 279.83,
  'temp_min': 282.64,
  'temp_max': 282.64,
  'pressure': 1015,
  'humidity': 60,
  'sea_level': 1015,
  'grnd_level': 930},
 'wind': {'speed': 1.65, 'deg': 255},
 'clouds': {'all': 97},
 'dt': 1593075303,
 'sys': {'country': 'CA', 'sunrise': 1593085378, 'sunset': 1593147391},
 'timezone': -25200,
 'id': 5881688,
 'name': 'Fort St. James',
 'cod': 200}

In [2]:
df = pd.read_json(decompress(get("http://bulk.openweathermap.org/sample/city.list.json.gz").content))
df

,id,name,state,country,coord
0,833,Ḩeşār-e Sefīd,,IR,"{'lon': 47.159401, 'lat': 34.330502}"
1,2960,‘Ayn Ḩalāqīm,,SY,"{'lon': 36.321911, 'lat': 34.940079}"
2,3245,Taglag,,IR,"{'lon': 44.98333, 'lat': 38.450001}"
3,3530,Qabāghlū,,IR,"{'lon': 46.168499, 'lat': 36.173302}"
4,5174,‘Arīqah,,SY,"{'lon': 36.48336, 'lat': 32.889809}"
...,...,...,...,...,...
209574,8277210,Beldibi,,TR,"{'lon': 30.707411, 'lat': 40.592731}"
209575,10722858,Forsytheganj,,BY,"{'lon': 29.733333000000002, 'lat': 52.633333}"
209576,71296900,Голубинка,,UA,"{'lon': 33.92, 'lat': 44.59}"
209577,102908597,Gerton,,US,"{'lon': -82.348056, 'lat': 35.479167}"


In [78]:
df[df['name']=='Abbotsford']

,id,name,state,country,coord
53912,2178377,Abbotsford,,AU,"{'lon': 151.133331, 'lat': -33.849998}"
157108,5243517,Abbotsford,WI,US,"{'lon': -90.315971, 'lat': 44.946362}"
161744,5881791,Abbotsford,,CA,"{'lon': -122.252571, 'lat': 49.05798}"
161745,5881792,Abbotsford,,CA,"{'lon': -122.285873, 'lat': 49.049679}"


In [6]:
def getData(city,countrycode):
    requestString='https://api.openweathermap.org/data/2.5/weather?q=%s,%s&APPID=%s' % (city,countrycode,key)
    
    res=requests.get(requestString)
    
    return res

### 1. Get hold of all the Canadian cities from the list of the cities.

In [3]:
canada = df[df['country']=='CA']
cities = canada['name'].head(30)

### 2. Then using all the data for the Canadian cities, push the values of [cityname, lon, lat, weather_main, weather_description, temp, humidity] into a pymongo database collection.

In [4]:
City=[]
Lon=[]
Lat=[]
Main=[]
Description=[]
Temp=[]
Humidity=[]
index=[]

for city in cities:
    country='ca'
    requestString='https://api.openweathermap.org/data/2.5/weather?q=%s,%s&APPID=%s' % (city,country,key)
    res=requests.get(requestString)
    mydict = res.json()
    
    City.append(mydict['name'])    
    Lon.append(mydict['coord']['lon'])    
    Lat.append(mydict['coord']['lat'])    
    Main.append(mydict['weather'][0]['main'])    
    Description.append(mydict['weather'][0]['description'])    
    Temp.append(mydict['main']['temp'])    
    Humidity.append(mydict['main']['humidity'])    
    index.append(mydict['id'])   
    

In [5]:
ca_weather = pd.DataFrame({'City':City, 'Lon':Lon, 'Lat':Lat, 'Main':Main, 'Description':Description, 'Temp':Temp, 'Humidity':Humidity, }, index=index)
ca_weather

,City,Lon,Lat,Main,Description,Temp,Humidity
5881688,Fort St. James,-124.26,54.46,Clouds,overcast clouds,283.62,66
5881774,Abbey,-108.75,50.73,Clouds,broken clouds,286.48,83
5881791,Abbotsford,-122.25,49.06,Clouds,scattered clouds,288.34,87
5881791,Abbotsford,-122.25,49.06,Clouds,scattered clouds,288.34,87
5881868,Aberdeen,-106.28,52.32,Clouds,scattered clouds,286.90,82
5881941,Abitibi-Témiscamingue,-77.78,48.10,Clouds,overcast clouds,287.15,87
5881954,Aboushagan Road,-64.35,45.98,Clouds,scattered clouds,286.80,87
5882033,Acadia Valley,-110.22,51.15,Clouds,broken clouds,290.93,59
5882100,Acme,-113.50,51.50,Clouds,overcast clouds,288.93,66
5882134,Acton,-80.04,43.63,Clouds,scattered clouds,288.01,63


In [143]:
data_dict = ca_weather.to_dict('records')

client=MongoClient('localhost', 27017)
weatherdb=client.APIweather
weathercol=weatherdb.APIweathercanada
weathercol.insert_many(data_dict)

In [2]:
client=MongoClient('localhost', 27017)
weatherdb=client.APIweather
weathercol=weatherdb.APIweathercanada

In [7]:
weathercol.count_documents({})

30

### 3. Extract the datetime in proper readable format.

In [8]:
dt=[]
for city in cities:
    country='ca'
    requestString='https://api.openweathermap.org/data/2.5/weather?q=%s,%s&APPID=%s' % (city,country,key)
    res=requests.get(requestString)
    mydict = res.json()
    dt.append(mydict['dt'])

In [25]:
for x in range(len(dt)):
    print(datetime.datetime.utcfromtimestamp(dt[x]).strftime('%Y-%m-%d %H:%M:%S'))

2020-06-26 08:52:05
2020-06-26 08:52:05
2020-06-26 08:50:28
2020-06-26 08:50:28
2020-06-26 08:52:06
2020-06-26 08:48:36
2020-06-26 08:52:07
2020-06-26 08:52:07
2020-06-26 08:49:54
2020-06-26 08:52:08
2020-06-26 08:52:08
2020-06-26 08:52:08
2020-06-26 08:52:09
2020-06-26 08:52:09
2020-06-26 08:49:52
2020-06-26 08:52:10
2020-06-26 08:52:10
2020-06-26 08:52:11
2020-06-26 08:52:11
2020-06-26 08:52:11
2020-06-26 08:52:12
2020-06-26 08:52:12
2020-06-26 08:52:12
2020-06-26 08:52:13
2020-06-26 08:52:13
2020-06-26 08:52:13
2020-06-26 08:52:14
2020-06-26 08:52:14
2020-06-26 08:52:14
2020-06-26 08:52:15


### 4. Sort the PyMongo Database collection by temp in descending order.

In [145]:
for i in weathercol.find().sort('Temp', -1):
    print(i)

{'_id': ObjectId('5ef46b80c5c8b5a108d3b1b1'), 'City': 'Albany', 'Lon': -65.05, 'Lat': 44.8, 'Main': 'Clouds', 'Description': 'overcast clouds', 'Temp': 295.27, 'Humidity': 88}
{'_id': ObjectId('5ef46b80c5c8b5a108d3b1a1'), 'City': 'Aboushagan Road', 'Lon': -64.35, 'Lat': 45.98, 'Main': 'Clouds', 'Description': 'overcast clouds', 'Temp': 293.92, 'Humidity': 94}
{'_id': ObjectId('5ef46b80c5c8b5a108d3b1b6'), 'City': 'Alberton South', 'Lon': -64.08, 'Lat': 46.8, 'Main': 'Clouds', 'Description': 'scattered clouds', 'Temp': 291.65, 'Humidity': 94}
{'_id': ObjectId('5ef46b80c5c8b5a108d3b1b5'), 'City': 'Alberton', 'Lon': -64.07, 'Lat': 46.82, 'Main': 'Clouds', 'Description': 'scattered clouds', 'Temp': 291.55, 'Humidity': 94}
{'_id': ObjectId('5ef46b80c5c8b5a108d3b19f'), 'City': 'Aberdeen', 'Lon': -106.28, 'Lat': 52.32, 'Main': 'Rain', 'Description': 'light intensity shower rain', 'Temp': 290.12, 'Humidity': 93}
{'_id': ObjectId('5ef46b80c5c8b5a108d3b1b8'), 'City': 'Albrights Corner', 'Lon': -6

In [146]:
weathercol.count_documents({})

30

### 5. Group the cities by their main weather and count the number of those cities which have similar weather.

In [161]:
agr = [ {'$group': {'_id': '$Main', 'count': { '$sum': 1 } } } ]

In [162]:
val=list(weathercol.aggregate(agr))
val

[{'_id': 'Rain', 'count': 4},
 {'_id': 'Clear', 'count': 6},
 {'_id': 'Clouds', 'count': 20}]